In [1]:
import pdfplumber

In [2]:

from langchain.schema import Document

# Add all your chapters here
pdf_files = ["ISLPcpt1,2.pdf", "ISLPcpt3.pdf", "ISLPcpt4.pdf", "ISLPcpt5.pdf", "ISLPcpt5.pdf"]

documents_plumber = []

for pdf_path in pdf_files:
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text:
                documents_plumber.append(
                    Document(page_content=text, metadata={"source": pdf_path, "page": i+1})
                )



In [3]:
print("pdfplumber:", len(documents_plumber), "pages total")
print(documents_plumber[0].page_content)


pdfplumber: 253 pages total
1
Introduction
An Overview of Statistical Learning
Statisticallearningreferstoavastsetoftoolsforunderstandingdata.These
tools can be classified as supervised or unsupervised. Broadly speaking,
supervised statistical learning involves building a statistical model for pre-
dicting, or estimating, an output based on one or more inputs. Problems of
thisnatureoccurinfieldsasdiverseasbusiness,medicine,astrophysics,and
public policy. With unsupervised statistical learning, there are inputs but
no supervising output; nevertheless we can learn relationships and struc-
ture from such data. To provide an illustration of some applications of
statistical learning, we briefly discuss three real-world data sets that are
considered in this book.
Wage Data
In this application (which we refer to as the Wage data set throughout this
book), we examine a number of factors that relate to wages for a group of
men from the Atlantic region of the United States. In particular, we wis

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
)

chunks = text_splitter.split_documents(documents_plumber)

print(f"Total chunks: {len(chunks)}")
print(chunks[0].page_content)


Total chunks: 1604
1
Introduction
An Overview of Statistical Learning
Statisticallearningreferstoavastsetoftoolsforunderstandingdata.These
tools can be classified as supervised or unsupervised. Broadly speaking,
supervised statistical learning involves building a statistical model for pre-
dicting, or estimating, an output based on one or more inputs. Problems of
thisnatureoccurinfieldsasdiverseasbusiness,medicine,astrophysics,and
public policy. With unsupervised statistical learning, there are inputs but


In [5]:
!pip install sentence-transformers

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")


# Example: Look at token length for a chunk
sample_text = chunks[0].page_content
tokens = tokenizer.tokenize(sample_text)
print(f"Tokens in chunk[0]: {len(tokens)}")


Tokens in chunk[0]: 108


In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/var/folders/8c/x37973bs11z07f191fqh8smr0000gn/T/ipykernel_99663/1728591684.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [ ]:
!pip install chromadb


In [8]:
from langchain.vectorstores import Chroma

# Save vector DB to a folder for reuse
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="chroma_db"
)

vectorstore.persist()  # Saves the DB
print("✅ Chunks embedded and stored in ChromaDB!")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


✅ Chunks embedded and stored in ChromaDB!


/var/folders/8c/x37973bs11z07f191fqh8smr0000gn/T/ipykernel_99663/3426439783.py:10: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()  # Saves the DB


In [17]:
pip install -U langchain langchain-community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/f6/d5/4861816a95b2f6993f1360cfb605aacb015506ee2090433a71de9cca8477/langchain-0.3.27-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-community from https://files.pythonhosted.org/packages/c8/bc/f8c7dae8321d37ed39ac9d7896617c4203248240a4835b136e3724b3bb62/langchain_community-0.3.27-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-text-splitters<1.0.0,>=0.3.9 from https://files.pythonhosted.org/packages/e2/52/7638394b88bc15083fd2c3752a843784d9d2d110d68fed6437c8607fb749/langchain_text_splitters-0.3.9-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.8
    Uninstalling langchain-text-splitter

In [10]:
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA

llm = ChatOllama(model="mistral", temperature=0)  # or llama3, gemma, etc.

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

query = "What is the difference between supervised and unsupervised learning?"
response = qa_chain(query)

print(" Answer:", response["result"])
print("\n Sources:")
for d in response["source_documents"]:
    print(f"- {d.metadata.get('source','?')} page {d.metadata.get('page','?')}")


 Answer:  In machine learning, supervised learning and unsupervised learning are two main categories of approaches.

1. Supervised Learning: In this type of learning, the algorithm learns from labeled data, which means that each example in the dataset has a known output or label. The goal is to learn a mapping function between the input (predictor measurements) and output (response). Examples include regression and classification problems.

2. Unsupervised Learning: In contrast, unsupervised learning deals with unlabeled data, where there are no predefined labels or outputs. The algorithm tries to find patterns, structures, or relationships within the data on its own. Clustering and dimensionality reduction are common examples of unsupervised learning techniques.

The context you provided discusses a scenario where an analysis might not be clearly supervised or unsupervised. This happens when we have more observations (n) than predictor measurements (m<n), and both predictor measuremen